# Modelagem Supervisionada: Regressão e Classificação

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/thomaschiari/ML_AI-Training/blob/main/M2-Modelling/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/thomaschiari/ML_AI-Training/blob/main/M2-Modelling/Models.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

### Autor:
- [Thomas Chiari Ciocchetti de Souza](https://github.com/thomaschiari)

In [29]:
import pandas as pd
import numpy as np
import warnings
import logging
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)

Vamos para a modelagem propriamente dita. Neste notebook, vamos abordar os seguintes tópicos:
- Processamento de dados utilizando Pipelines
- O que é aprendizado supervisionado?
- Problemas de regressão e classificação
- Principais algoritmos de regressão e avaliação de modelos
- Principais algoritmos de classificação e avaliação de modelos
- Testes estatísticos e escolha de modelos
- Otimização de hiperparâmetros

## 1. Processamento de dados utilizando Pipelines

O pipeline é uma ferramenta que permite encadear transformações de dados e modelos de aprendizado de máquina. Ele é muito útil para automatizar o processamento de dados e a criação de modelos, além de facilitar a reprodutibilidade dos resultados. É usado para automatizar os fluxos de trabalho e facilitar o processo de modelagem. Aqui, vamos utilizar as Pipelines do `sklearn` para automatizar processamento de dados que foi realizado no notebook do módulo anterior.

In [30]:
# Importando os dados
import os
import urllib.request
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/thomaschiari/ML_AI-Training/main/M1-Introduction/data/house-prices/"
TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"
TXT_PATH = "data_description.txt"
TRAIN_URL = DOWNLOAD_ROOT + TRAIN_PATH
TEST_URL = DOWNLOAD_ROOT + TEST_PATH
TXT_URL = DOWNLOAD_ROOT + TXT_PATH
DATA_PATH = os.path.join("data", "house-prices")

def fetch_data(data_url, data_path):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    csv_path = os.path.join(data_path, data_url.split('/')[-1])
    if not os.path.isfile(csv_path):
        urllib.request.urlretrieve(data_url, csv_path)

fetch_data(TRAIN_URL, DATA_PATH)

In [31]:
PATH_TRAIN = os.path.join('data', 'house-prices', 'train.csv')
PATH_TEST = os.path.join('data', 'house-prices', 'test.csv')

In [32]:
dataset = pd.read_csv(PATH_TRAIN, encoding='utf-8')
dataset_test = pd.read_csv(PATH_TEST, encoding='utf-8')
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [33]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


Em uma Pipeline, podemos usar os transformadores padrão do `scikit-learn`, como o `StandardScaler` e o `OneHotEncoder`, e também podemos usar os nossos próprios transformadores personalizados. Para isso, basta criar uma classe que herda da classe `BaseEstimator` e implementar os métodos `fit()` e `transform()`. Vamos criar um transformador personalizado para realizar a transformação logarítmica, como foi realizado no notebook anterior. Vamos também criar um transformador para realizar a binarização e categorização de variáveis com alta concentração de zeros, assim como foi feito no notebook anterior.

In [34]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [35]:
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if self.columns is None:
            self.columns = X.columns
        for col in self.columns:
            X[col] = np.log1p(X[col])
        return X

In [36]:
class BinaryTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        if self.columns is None:
            self.columns = X.columns
        for col in self.columns:
            X[col] = X[col].apply(lambda x: 1 if x > 0 else 0)
        return X

In [37]:
class BinsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, bins=10, labels=False):
        self.columns = columns
        self.bins = bins
        self.labels = labels
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        if self.columns is None:
            self.columns = X.columns
        for col in self.columns:
            X[col] = pd.cut(X[col], bins=self.bins, labels=self.labels)
        return X

No `scikit-learn`, temos tanto o Pipeline quanto o Column Transformer. Os dois diferem na forma como lidam com as colunas. O Pipeline aplica as transformações em todas as colunas, enquanto o Column Transformer aplica as transformações apenas nas colunas especificadas. O interessante é que podemos utilizar o Column Transformer dentro de uma Pipeline, automatizando todo o processamento de dados, selecionando as colunas a serem transformadas com cada método. Vamos, agora, de acordo com análise exploratória realizada no notebook anterior, selecionar uma lista de variáveis para cada método de transformação. Vamos, também, criar uma lista de variáveis categóricas que serão transformadas de acordo com o método de codificação escolhido. Por fim, vamos aplicar um método de transformação de `StandardScaler` em todas as variáveis numéricas. Isso garante que todas as variáveis estejam na mesma escala, o que pode ser importante em alguns tipos de modelo. 

In [38]:
drop = ['Id', 'PoolArea', 'GarageArea']

dataset.drop(drop, axis=1, inplace=True)

In [39]:
y = dataset['SalePrice']
X = dataset.drop(['SalePrice'], axis=1)

log_features = ['1stFlrSF', 'TotalBsmtSF', 'BsmtUnfSF', 'LotArea', 'LotFrontage']

binary = ['ScreenPorch', '3SsnPorch', 'EnclosedPorch', 'LowQualFinSF']
bins = ['OpenPorchSF', 'WoodDeckSF']

df_cat = X.select_dtypes(include=['object'])
df_num = X.select_dtypes(exclude=['object'])
var_cat = df_cat.columns
var_num = df_num.columns

In [40]:
num_pipeline = Pipeline([
    ('inputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

full_pipeline = ColumnTransformer([
    #('inpute_cat', SimpleImputer(strategy='most_frequent'), var_cat),
    ('log', LogTransformer(), log_features),
    ('cat', OneHotEncoder(drop='first'), var_cat),
    ('binary', BinaryTransformer(), binary),
    ('bins', BinsTransformer(bins=5), bins),
    ('num', num_pipeline, var_num),
])

full_pipeline.fit(X)
X_prepared = full_pipeline.transform(X)
small_constant = 1e-10 
y_prepared = np.log(y + small_constant)


In [41]:
X_prepared
df = pd.DataFrame(X_prepared)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,259,260,261,262,263,264,265,266,267,268
0,6.753438,6.753438,5.017280,9.042040,4.189655,0.0,0.0,1.0,0.0,1.0,...,1.017598,0.311725,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.087688,-1.599111,0.138777
1,7.141245,7.141245,5.652489,9.169623,4.394449,0.0,0.0,1.0,0.0,1.0,...,-0.107927,0.311725,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.087688,-0.489110,-0.614439
2,6.825460,6.825460,6.075346,9.328212,4.234107,0.0,0.0,1.0,0.0,1.0,...,0.934226,0.311725,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.087688,0.990891,0.138777
3,6.869014,6.629363,6.293419,9.164401,4.110874,0.0,0.0,1.0,0.0,1.0,...,0.809167,1.650307,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.087688,-1.599111,-1.367655
4,7.044033,7.044033,6.196444,9.565284,4.442651,0.0,0.0,1.0,0.0,1.0,...,0.892540,1.650307,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.087688,2.100892,0.138777


In [42]:
df_nan = df.isnull().sum().sort_values(ascending=False)
cols = df_nan[df_nan > 0].index
df[cols] = df[cols].fillna(df[cols].mean())
X_prepared = df.to_numpy()

In [43]:
y_prepared

0       12.247694
1       12.109011
2       12.317167
3       11.849398
4       12.429216
          ...    
1455    12.072541
1456    12.254863
1457    12.493130
1458    11.864462
1459    11.901583
Name: SalePrice, Length: 1460, dtype: float64

**Sobre o `Standard Scaler`**

O Standard Scaler se trata de uma das técnicas de padronização de dados numéricos. Muitas vezes, tais dados são apresentados em escalas muito diferentes. Por exemplo, dentro do próprio dataset que estamos utilizando, há uma variável para o número de quartos, que pode variar de 1 a 10 normalmente, e outra variável para a área total da casa em Square Feet, que possui valores de grandeza maior, na casa dos milhares. Em modelos lineares, o peso atribuído a cada feature, como será visto em sequência, impede que isso seja um problema; a magnitude da feature será compensada por um peso menor ou maior, a ser atribuído pelo modelo. Contudo, em outros modelos, é importante que estejam todas as features em escalas próximas. 
O Standard Scaler irá realizar uma normalização dos dados. O cálculo que realiza é o seguinte:
$$
z = \frac{x - \mu}{\sigma}
$$
Em que $x$ é o valor da feature, $\mu$ é a média da feature e $\sigma$ é o desvio padrão da feature. O resultado é que os dados serão normalizados em torno de 0, com desvio padrão 1. Acima, o dataset preparado `X_prepared` está apresentado de acordo com essas características. 

## 2. O que é Aprendizado Supervisionado?

Aprendizado supervisionado é o método mais comum de Machine Learning. Quando recebemos uma base de dados para treinar um modelo supervisionado, todas as observações possuem uma resposta esperada. Ou seja, além das features com as quais desejamos treinar o modelo, temos também a variável resposta, que é o que queremos prever. No caso que estamos utilizando aqui, a variável resposta é o preço das casas, que está presente no dataset e queremos prevê-la usando um conjunto de features. Portanto, se trata de um problema de aprendizado de máquina supervisionado e, mais especificamente, de regressão.

## 3. Problemas de Regressão e Classificação

A diferença entre problemas de regressão e classificação é o que desejamos prever. Em problemas de regressão, desejamos prever um valor contínuo; ou seja, um valor que pode assumir qualquer valor dentro de um intervalo. No caso do dataset que estamos utilizando, desejamos prever o preço das casas, que pode assumir qualquer valor dentro do intervalo de valores reais. Em problemas de classificação, desejamos prever um valor discreto; ou seja, um valor que pode assumir apenas um conjunto de valores finitos. Por exemplo, se desejamos prever se um e-mail é spam ou não, estamos realizando um problema de classificação. A resposta esperada é apenas "sim" ou "não". Se desejamos prever se uma pessoa é boa ou má pagadora de crédito, estamos realizando um problema de classificação. A resposta esperada é apenas "sim" ou "não". Existem problemas de classificação binários, com apenas duas respostas possíveis, e problemas multiclasse, em que há mais possibilidades de classificação.

## 4. Regressão e os Principais Algoritmos

Existem vários modelos de regressão, para realizar predições com valores contínuos. Aqui, vamos tratar dos principais e mais utilizados, tanto em competições de Machine Learning quanto em aplicações práticas.

### 4.1. Regressão Linear

Uma regressão linear é o modelo mais básico, que vale a pena ser testado em qualquer problema de regressão. A regressão linear é um modelo linear, ou seja, que assume que a relação entre as variáveis é linear. A equação de uma regressão linear múltipla é a seguinte:
$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n
$$
Com os valores de $\beta$ sendo os coeficientes do modelo, ou seja, os pesos, e $\beta_0$ sendo o termo de viés. A regressão linear é um modelo simples, que não possui muitos parâmetros para serem otimizados. Por isso, é um modelo rápido de ser treinado e que pode ser utilizado como baseline para comparação com outros modelos. Porém, muitas vezes, pode apresentar uma solução suficientemente boa para o problema.

Em todo problema de Machine Learning, as amostras de entrada serão caracterizadas como um vetor de features, que pode ser caracterizado como:
$$
\mathbf{X} = [x_1, x_2, ..., x_n]
$$
E um modelo de machine larning será uma função que permite estimar a saída para esse vetor de entrada. Essa saída vai depender dos parâmetros do modelo que serão treinados com os dados de treino. Ou seja:
$$
\hat{y} = f(\mathbf{X}, \theta)
$$
Com $\theta$ sendo o vetor de parâmetros do modelo. No caso da regressão linear, os parâmetros são os coeficientes $\beta$ e o termo de viés $\beta_0$. Para termos de notação, $\hat{y}$ é a saída estimada pelo modelo e $y$ é a saída real, que está presente no dataset. O objetivo do modelo é minimizar o erro entre a saída estimada e a saída real. Para isso, é utilizado um método de otimização, que pode ser o método dos mínimos quadrados, por exemplo. O método dos mínimos quadrados consiste em minimizar a soma dos quadrados dos erros, ou seja:
$$
\min_{\theta} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$
Isso quer dizer que, basicamente, o modelo irá tentar minimizar a soma dos quadrados dos erros, constituídos na diferença entre a saída real e a estimada. 

A regressão linear será treinada podendo utilizar tanto conceitos de álgebra linear, quanto conceitos de matemática multivariada.

**Gradiente Descendente:** o gradiente descendente irá atualizar os parâmetros de acordo com a seguinte equação:
$$
\theta_{i+1} = \theta_i - \alpha \frac{\partial}{\partial \theta_i} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$
Ou seja, para cada etapa do modelo, o algoritmo irá derivar a função de erro e irá atualizar os parâmetros de acordo com a direção do gradiente. O parâmetro $\alpha$ é o learning rate, que controla o tamanho do passo que o algoritmo irá dar em cada etapa. Se o learning rate for muito pequeno, o algoritmo irá demorar muito para convergir. Se o learning rate for muito grande, o algoritmo pode não convergir. O gradiente descendente é um algoritmo iterativo, que irá atualizar os parâmetros até que a função de erro convirja para um mínimo local. O gradiente descendente é um algoritmo muito utilizado para treinar modelos de regressão linear, mas também pode ser utilizado para treinar outros modelos de regressão e classificação.

**Equação Normal:** a equação normal é uma equação que permite encontrar os parâmetros do modelo de regressão linear sem a necessidade de utilizar um algoritmo iterativo. A equação normal é a seguinte:
$$
\theta = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$
Em que $\mathbf{X}$ é a matriz de features, $\mathbf{y}$ é o vetor de saídas e $\theta$ é o vetor de parâmetros do modelo. A equação normal é uma equação matemática que permite encontrar os parâmetros do modelo de regressão linear de forma direta. Porém, ela só pode ser utilizada se a matriz $\mathbf{X}^T \mathbf{X}$ for inversível. Caso contrário, o algoritmo irá apresentar um erro. Além disso, a equação normal não é um algoritmo iterativo, ou seja, não é possível atualizar os parâmetros do modelo de forma incremental. Porém, ela é muito mais rápida que o gradiente descendente, pois não é necessário realizar várias iterações para encontrar os parâmetros do modelo.

Para explicar com precisão como um modelo de regressão linear é treinado, precisamos de muita álgebra linear. O importante é saber que o modelo de regressão linear é um modelo simples, que pode ser treinado de forma rápida e que pode ser utilizado como baseline para comparação com outros modelos. Porém, ele assume que a relação entre as variáveis é linear, o que pode não ser verdade em muitos casos. Também pode ser interessante que a variável resposta se aproxime de uma distribuição normal, o que pode não ser verdade em muitos casos. Porém, é um modelo que vale a pena ser testado em qualquer problema de regressão. Para mais informações sobre o modelo de regressão linear e para tentar realizar uma implementação do modelo com força bruta, recomendo acessar a parte 4 do módulo de Pandas. 

O primeiro passo para realizar o modelo é separar em variáveis de entrada e variável de saída. Para isso, vamos utilizar o dataset preparado `X_prepared` e a variável resposta `y`. Vamos, também, separar os dados em treino e teste, para que possamos avaliar o modelo. Vamos utilizar a função `train_test_split` do `scikit-learn` para realizar essa separação. Vamos utilizar 20% dos dados para teste e 80% para treino. Vamos, também, utilizar o parâmetro `random_state` para garantir que os dados serão separados da mesma forma em todas as execuções do notebook. É importante realizarmos essa separação para que possamos avaliar o modelo em dados que não foram utilizados para treiná-lo. Caso contrário, o modelo pode apresentar um desempenho muito bom nos dados de treino, mas um desempenho ruim nos dados de teste. Isso é chamado de overfitting, que é quando o modelo se ajusta muito bem aos dados de treino, mas não consegue generalizar para dados novos.

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X_prepared, y_prepared, test_size=0.2, random_state=42)

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

LinearRegression()

In [45]:
y_pred = lin_reg.predict(X_test)
y_pred

array([ 1.19858353e+01,  1.27412781e+01,  1.15330261e+01,  1.20711624e+01,
        1.26526371e+01,  1.12373718e+01,  1.24404610e+01,  1.19570984e+01,
        1.12790661e+01,  1.18716133e+01,  1.19368168e+01,  1.16082035e+01,
        1.11020821e+01,  1.22373718e+01,  1.20234509e+01,  1.18173962e+01,
        1.21145153e+01,  1.17695446e+01,  1.16362438e+01,  1.22614880e+01,
        1.18232376e+01,  1.22327152e+01,  1.20193363e+01,  1.18360046e+01,
        1.21799808e+01,  1.19338692e+01,  1.21418591e+01,  1.15493835e+01,
        1.20967110e+01,  1.22273083e+01,  1.21181595e+01,  1.25202950e+01,
        1.23249645e+01,  1.16838692e+01,  1.24665661e+01,  1.19409493e+01,
        1.18711429e+01,  1.22253552e+01,  1.26674759e+01,  1.16362618e+01,
        1.17404740e+01,  1.22543396e+01,  1.16418591e+01,  1.27141605e+01,
        1.17419567e+01,  1.17204724e+01,  1.15810501e+01,  1.18144665e+01,
        1.30168949e+01,  1.18479138e+01,  1.17136364e+01,  1.22698246e+01,
        1.15854805e+01,  

Como realizamos uma transformação logarítmica na variável resposta, nosso vetor de previsões $\hat{y}$ irá apresentar a previsão do logaritmo do preço das casas. Para obtermos a previsão do preço das casas, precisamos aplicar a função exponencial no vetor de previsões.

Vamos avaliar o modelo de regressão linear. Para isso, vamos usar o RMSE, que é a raiz do erro quadrático médio. O RMSE é uma métrica de avaliação de modelos de regressão que mede a diferença entre os valores reais e os valores previstos pelo modelo. O RMSE é calculado da seguinte forma:
$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$
Como estão em escala logarítmica, vamos calcular o RMSE em escala logarítmica e depois aplicar a função exponencial para obtermos o RMSE em escala original. Vamos também calcular o coeficiente de determinação $R^2$, que é uma métrica que mede a proporção da variância da variável resposta que é explicada pelo modelo. O $R^2$ é calculado da seguinte forma:
$$
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$
Em que $\bar{y}$ é a média da variável resposta. O $R^2$ é uma métrica que varia de 0 a 1. Quanto mais próximo de 1, melhor o modelo. Quanto mais próximo de 0, pior o modelo. O $R^2$ é uma métrica que pode ser utilizada para comparar modelos. Porém, não é uma métrica que pode ser utilizada para avaliar a qualidade de um modelo. Isso porque o $R^2$ sempre irá aumentar quando adicionamos mais variáveis ao modelo, mesmo que essas variáveis não sejam relevantes para o modelo.

In [46]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred_orig, y_test_orig = np.exp(y_pred), np.exp(y_test)
y_pred_orig[y_pred_orig == np.inf] = np.nan
y_pred_orig = np.nan_to_num(y_pred_orig)
y_test_orig[y_test_orig == np.inf] = np.nan
y_test_orig = np.nan_to_num(y_test_orig)

rmse_lr = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
r2_lr = r2_score(y_test_orig, y_pred_orig)

print('RMSE: ', rmse_lr)
print('R2: ', r2_lr)

RMSE:  34039.85543761642
R2:  0.8489357913972281


In [47]:
print(f'Preço médio das casas: {y.mean():.2f}.')
print(f'RMSE representa aproximadamente {rmse_lr/y.mean()*100:.2f}% do preço médio de casas.')

Preço médio das casas: 180921.20.
RMSE representa aproximadamente 18.81% do preço médio de casas.


Como podemos ver, temos um RMSE de aproximadamente 35000 e um R2 score de aproximadamente 0.85. Como dito anteriormente, o R2 score não é utilizado para avaliar a qualidade do modelo, mas sim o quanto do resultado é explicado pelo modelo, sendo muito mais útil para realizar comparações entre modelos. O RMSE já consegue avaliar a qualidade do modelo, mas é uma métrica que depende da escala da variável resposta. Vamos comparar o resultado obtido com a média do preço das casas, que é de aproximadamente 180000. O RMSE representa aproximadamente 20% da média do preço das casas. Para casas com preço maior, isso pode ser um bom resultado, mas para casas com preço menor pode representar um erro de mais de 50%. Vamos utilizar outros modelos para compará-los.

### 4.2. Regressão de Árvore de Decisão e Floresta Aleatória

Uma árvore de decisão é exatamente o que o nome sugere: uma árvore que, a partir da raiz, a cada nó, toma uma decisão até chegar no melhor valor. Mais para a frente vamos ver um exemplo de um regressor, e plotar a árvore. Por enquanto, vamos entender como funciona o algoritmo.

A árvore de decisão começa em um nó raiz, contendo todo o conjunto de dados. Para cada nó, o algoritmo irá escolher uma feature que melhor separa os dados, com base em algum critério. O critério mais comum para problemas de classificação é a impureza de Gini, e para problemas de regressão, o erro quadrático médio. O algoritmo tomará uma decisão e dividirã a amostra nos dois nós filhos. Cada nó irá representar uma regra de decisão. Isso ficará mais claro quando plotarmos a árvore visualmente. O algoritmo irá repetir esse processo até que não seja mais possível dividir os dados, ou até chegar na profundidade máxima (critério de parada).

O algoritmo de Random Forest (Floresta Aleatória) funciona de forma semelhante, porém se trata de um modelo de ensemble: vários modelos de árvore de decisão são utilizados sequencialmente para tomar uma decisão. Cada árvore é construída de forma independente usando parte aleatória da amostra de treino, e cada árvore é treinada de forma independente, seguindo o mesmo algoritmo descrito anteriormente. Em problemas de classificação, a decisão final será com base em uma votação entre todas as árvores, com a classe mais votada sendo o retorno do algoritmo. Em problemas de regressão, o resultado final é a média de todas as árvores da floresta. 

A Floresta Aleatória é um dos algoritmos mais utilizados em problemas de classificação, mas também é importante em problemas de regressão. É um algoritmo que apresenta um bom desempenho na maioria dos problemas, com vantagens frente às Árvores de Decisão individuais por reduzir o Overfitting (conceito que trataremos mais adiante). Caso queira saber mais sobre os modelos de árvore, consulte nossa bibliografia complementar!

Para plotar uma árvore de decisão básica com os dados que temos, acesse o notebook `plot_tree.ipynb`, disponível na mesma pasta desse notebook. Aqui, vamos partir para o modelo e avaliação.

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y_prepared, test_size=0.2, random_state=42)

In [49]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)

tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [50]:
y_pred = tree_reg.predict(X_test)

In [51]:
y_pred_orig, y_test_orig = np.exp(y_pred), np.exp(y_test)
y_pred_orig[y_pred_orig == np.inf] = np.nan
y_pred_orig = np.nan_to_num(y_pred_orig)
y_test_orig[y_test_orig == np.inf] = np.nan
y_test_orig = np.nan_to_num(y_test_orig)

rmse_dt = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
r2_dt = r2_score(y_test_orig, y_pred_orig)

print('RMSE: ', rmse_dt)
print('R2: ', r2_dt)

RMSE:  43530.37832010858
R2:  0.7529578553104747


Seguindo a lógica explicada anteriormente, o modelo de árvore de decisão chegou a um resultado com RMSE de aproximadamente 43500, e R2 Score de aproximadamente 0.75. Em comparação com o modelo linear, a árvore de decisão apresentou um desempenho pior, porém rodou de forma muito mais eficiente. Vamos, agora, testar os resultados para o modelo de Floresta Aleatória.

In [52]:
from sklearn.ensemble import RandomForestRegressor

# o hiperparâmetro n_estimators define o número de árvores na floresta
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)

forest_reg.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [53]:
y_pred = forest_reg.predict(X_test)

In [54]:
y_pred_orig, y_test_orig = np.exp(y_pred), np.exp(y_test)
y_pred_orig[y_pred_orig == np.inf] = np.nan
y_pred_orig = np.nan_to_num(y_pred_orig)
y_test_orig[y_test_orig == np.inf] = np.nan
y_test_orig = np.nan_to_num(y_test_orig)

rmse_rf = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
r2_rf = r2_score(y_test_orig, y_pred_orig)

print('RMSE: ', rmse_rf)
print('R2: ', r2_rf)

RMSE:  29085.41396725085
R2:  0.8897098659436827


In [55]:
print('Preço médio das casas: {:.2f}.'.format(y.mean()))
print('RMSE representa aproximadamente {:.2f}% do preço médio de casas.'.format(rmse_rf/y.mean()*100))

Preço médio das casas: 180921.20.
RMSE representa aproximadamente 16.08% do preço médio de casas.


O modelo de Floresta Aleatória foi o que, até agora, apresentou o melhor desempenho, porém também foi o que mais demorou para rodar. Apresentou um RMSE de menos de 30000 e R2 Score de 0.89. O erro representa cerca de 16% do preço médio das casas, o que é um desempenho relevante. Contudo, com um trade-off de tempo de processamento.

Vamos ver mais alternativas de modelos de regressão.

### 4.3. Regressão de KNN

O algoritmo K-Nearest Neighbors (K-Vizinhos mais próximos) ou, abreviado, KNN, estima os valores alvo com base na média dos valores alvo das $K$ instâncias mais próximas no espaço de características. Simples assim.

O algoritmo irá calcular a distância entre as instâncias de treino e a instância de teste, e irá selecionar as $K$ instâncias mais próximas. A distância pode ser calculada de várias formas, sendo a distância euclidiana a mais comum, mas também podendo utilizar a distância de Manhattan, por exemplo. É um algoritmo muito simples, que pode ser muito útil em alguns casos, mas com muitas features e muitos vizinhos pode apresentar um processamento muito lento.

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y_prepared, test_size=0.2, random_state=42)

In [67]:
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor(n_neighbors=5)

knn_reg.fit(X_train, y_train)

KNeighborsRegressor()

In [68]:
y_pred = knn_reg.predict(X_test)

In [69]:
y_pred_orig, y_test_orig = np.exp(y_pred), np.exp(y_test)
y_pred_orig[y_pred_orig == np.inf] = np.nan
y_pred_orig = np.nan_to_num(y_pred_orig)
y_test_orig[y_test_orig == np.inf] = np.nan
y_test_orig = np.nan_to_num(y_test_orig)

rmse_knn = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
r2_knn = r2_score(y_test_orig, y_pred_orig)

print('RMSE: ', rmse_knn)
print('R2: ', r2_knn)

RMSE:  38111.9946563933
R2:  0.8106307218944703


É um modelo muito simples e que possui várias aplicações, porém não possui, nesse caso, um desempenho muito bom.

### 4.4. Regressão de Gradiente

Algoritmos de Gradient Boosting são modelos de Ensemble, como as florestas aleatórias, que utilizam vários modelos fracos para formar um modelo forte, basicamente. O Gradient Boosting funciona de maneira iterativa, ajustando um novo modelo fraco, geralmente árvores de decisão pequenas, para corrigir os erros de modelos anteriores. O algoritmo irá ajustar um modelo inicial, e irá calcular os erros residuais. O próximo modelo irá tentar corrigir esses erros residuais, e assim por diante. O algoritmo irá parar quando atingir um número máximo de iterações, ou quando o erro não puder ser mais reduzido.

O algoritmo XGBoost (Extreme Gradient Boosting) implementa métodos de regularização no processo de otimização, tornando o modelo mais robusto contra overfitting. Em resumo, regularização é um processo dentro do modelo que penaliza modelos com muitos parâmetros, evitando o sobreajuste (vamos tratar mais disso mais para a frente). O XGBoost é um dos algoritmos mais utilizados em competições de Machine Learning, e é um dos algoritmos mais poderosos para problemas de regressão e classificação.

O algoritmo LoghtGBM é outro exemplo de algoritmo de gradiente, que utiliza uma técnica chamada "Gradient-based One-Side Sampling" (GOSS). No geral, não é muito importante entendermos como eesa técnica funciona, mas é importante saber que ela torna o algoritmo mais rápido e mais eficiente. 

Em resumo, os três algoritmos são muito importantes e é sempre relevante testá-los em problemas de regressão e classificação.

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y_prepared, test_size=0.2, random_state=42)

In [89]:
# Utilizando Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor

gbrt_reg = GradientBoostingRegressor(max_depth=3, n_estimators=100, learning_rate=1.0, random_state=42)

gbrt_reg.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=1.0, random_state=42)

In [90]:
y_pred = gbrt_reg.predict(X_test)

In [91]:
y_pred_orig, y_test_orig = np.exp(y_pred), np.exp(y_test)
y_pred_orig[y_pred_orig == np.inf] = np.nan
y_pred_orig = np.nan_to_num(y_pred_orig)
y_test_orig[y_test_orig == np.inf] = np.nan
y_test_orig = np.nan_to_num(y_test_orig)

rmse_gbrt = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
r2_gbrt = r2_score(y_test_orig, y_pred_orig)

print('RMSE: ', rmse_gbrt)
print('R2: ', r2_gbrt)

RMSE:  32516.53488661677
R2:  0.8621538364246036


In [94]:
# Utilizando XGBoost
!pip install xgboost --quiet

In [116]:
import xgboost as xgb
import multiprocessing

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, 
                           n_jobs=multiprocessing.cpu_count(),
                           n_estimators=500, 
                           max_depth=3, 
                           learning_rate=0.1)

xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=8,
             num_parallel_tree=None, random_state=42, ...)

In [117]:
y_pred = xgb_reg.predict(X_test)

In [118]:
y_pred_orig, y_test_orig = np.exp(y_pred), np.exp(y_test)
y_pred_orig[y_pred_orig == np.inf] = np.nan
y_pred_orig = np.nan_to_num(y_pred_orig)
y_test_orig[y_test_orig == np.inf] = np.nan
y_test_orig = np.nan_to_num(y_test_orig)

rmse_xgb = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
r2_xgb = r2_score(y_test_orig, y_pred_orig)

print('RMSE: ', rmse_xgb)
print('R2: ', r2_xgb)

RMSE:  25873.047896095453
R2:  0.9127267094838281


In [119]:
# Utilizando LightGBM
!pip install lightgbm --quiet

In [121]:
import lightgbm as lgb

lgb_reg = lgb.LGBMRegressor(objective='regression', random_state=42,
                            n_jobs=multiprocessing.cpu_count())

lgb_reg.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061817 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4054
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 179
[LightGBM] [Info] Start training from score 12.030652


LGBMRegressor(n_jobs=8, objective='regression', random_state=42)

In [122]:
y_pred = lgb_reg.predict(X_test)

In [123]:
y_pred_orig, y_test_orig = np.exp(y_pred), np.exp(y_test)
y_pred_orig[y_pred_orig == np.inf] = np.nan
y_pred_orig = np.nan_to_num(y_pred_orig)
y_test_orig[y_test_orig == np.inf] = np.nan
y_test_orig = np.nan_to_num(y_test_orig)

rmse_lgb = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
r2_lgb = r2_score(y_test_orig, y_pred_orig)

print('RMSE: ', rmse_lgb)
print('R2: ', r2_lgb)

RMSE:  29264.9326114545
R2:  0.8883442167100417


Aqui temos alguns dos modelos mais poderosos que testamos até agora, e por isso são os modelos mais amplamente utilizados em problemas e competições de Machine Learning. Quando os problemas não incluem questões complexas que só seriam resolvidas com Deep Learning, provavelmente são os modelos que se saem melhor. É sempre válido testá-los em problemas de regressão e classificação.

### 4.5. Support Vector Machine

SVMs normalmente são usadas para problemas de classificação, mas que também podem ser adaptadas para problemas de regressão. Nesse caso, é chamado de SVR (Support Vector Regression). O algoritmo irá tentar encontrar a função que melhor se ajusta aos dados. Diferente de modelos lineares, a SVR funciona muito bem para dados que não seguem distribuições lineares, que seguem distribuições complexas. Ou seja, funciona bem quando existe uma relação complexa e não linear entre as variáveis independentes e a dependente. 

Começamos o modelo selecionando um Kernel (uma função de mapeamento) que será responsável por mapear os dados e treinar o modelo. O Kernel mais comum é o RBF (Radial Basis Function), mas também podemos utilizar o Linear, o Polynomial, o Sigmoid, entre outros. É interessante testar, de acordo com a distribuição dos seus dados. O SVR vai realizar o treinamento de modo a encontrar a função que minimiza a diferença entre as previsões e os valores reais.

O SVR possui um hiperparâmetro de regularização: `C`. Como já visto anteriormente, a regularização é importante para evitar o sobreajuste no modelo, e o parâmetro `C`, basicamente, representa o ajuste da penalização por erros de treinamento. Vamos tratar melhor sobre isso mais para a frente, nos tópicos de regularização.

Vamos testar o modelo de SVR o Kernel mais utilizado.

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y_prepared, test_size=0.2, random_state=42)

In [125]:
from sklearn.svm import SVR

svr_reg = SVR(kernel='rbf', C=100000)

svr_reg.fit(X_train, y_train)

SVR(C=100000)

In [126]:
y_pred = svr_reg.predict(X_test)

In [127]:
y_pred_orig, y_test_orig = np.exp(y_pred), np.exp(y_test)
y_pred_orig[y_pred_orig == np.inf] = np.nan
y_pred_orig = np.nan_to_num(y_pred_orig)
y_test_orig[y_test_orig == np.inf] = np.nan
y_test_orig = np.nan_to_num(y_test_orig)

rmse_svr = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
r2_svr = r2_score(y_test_orig, y_pred_orig)

print('RMSE: ', rmse_svr)
print('R2: ', r2_svr)

RMSE:  27458.688715756904
R2:  0.9017017574451587
